In [1]:

import tabula
import pandas as pd
from pyswip import Prolog
from unidecode import unidecode


In [2]:
lista_tabelas = tabula.read_pdf('historico_hellen.pdf', pages = ['all'], multiple_tables= True)
print(len(lista_tabelas))

5


In [3]:
list_columns = ['Ano/Periodo Letivo', 'Componente', 'Codigo', 'Nome', 'CH', 'Turma', 'Freq %', 'Nota', 'Situacao']

In [4]:
tabelas = []

for i in lista_tabelas:
    componente = (i.iloc[0] == 'Componente Curricular').any()
    nota = (i.iloc[0] == 'Nota').any()
    if componente and nota:
        print(len(i.columns))
        display(i.head(8))
        tabelas.append(i)
    else:
        print('Tabela não é do histórico')

9


,Ano/Período,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,Componente Curricular,CH,Turma,Freq %,Nota,Situação
1,Letivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,INGRESSANTE - REGULAR: NÃO HABILITADO AO ENADE...,NaN,NaN,NaN,NaN,NaN
3,2020.1,NaN,ENADE,NaN,0,--,--,---,--
4,NaN,NaN,NaN,NATUREZA DO PROJETO PEDAGÓGICO DO CURSO,NaN,NaN,NaN,NaN,NaN
5,2020.2,*,CIC0004,ALGORITMOS E PROGRAMAÇÃO DE COMPUTADORES,90,13A,"95,0",MS,APR
6,2020.2,NaN,FGA0161,ENGENHARIA E AMBIENTE,60,B,"100,0",MS,APR
7,2020.2,NaN,FGA0163,INTRODUÇÃO À ENGENHARIA,30,B,"100,0",MS,APR


10


,Ano/Período,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,NaN,NaN,NaN,Componente Curricular,CH,Turma,Freq %,Nota,Situação
1,Letivo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021.2,NaN,MAT0026,CÁLCULO 2,NaN,90,04A,"100,0",MS,APR
3,2022.1,NaN,FGA0108,MATEMÁTICA DISCRETA 2,NaN,60,01,"96,0",MS,APR
4,2022.1,NaN,FGA0138,MÉTODOS DE DESENVOLVIMENTO DE SOFTWARE,NaN,60,01,"93,0",MM,APR
5,2022.1,NaN,FGA0147,ESTRUTURA DE DADOS E ALGORITMOS,NaN,60,01,"98,0",MM,APR
6,2022.1,NaN,FGA0160,MÉTODOS NUMÉRICOS PARA ENGENHARIA,NaN,60,04,"100,0",MS,APR
7,2022.1,*,FGA0174,DESENVOLVIMENTO SUSTENTÁVEL,NaN,60,01,"96,0",MS,APR


Tabela não é do histórico
Tabela não é do histórico
Tabela não é do histórico


In [ ]:
situacao_aceitas = ['CUMP', 'APR', 'MATR']
colunas_fixas = ['Codigo', 'Nome']
tabelas_formatadas = []

for tabela in tabelas:
    try:
        if len(tabela.columns) == 10:
            print(len(tabela.columns))
            del tabela['Unnamed: 3']
        
        tabela.drop([0, 1], inplace=True)
        tabela.columns = list_columns
        
        tabela = tabela[tabela['Situacao'].isin(situacao_aceitas)]
        
        tabela = tabela[colunas_fixas]
        tabela.reset_index(drop=True, inplace=True)
        #display(tabela)
        tabelas_formatadas.append(tabela)
    except Exception as e:
        print('Erro: ', e)
        pass
    
tabela_final = pd.concat(tabelas_formatadas, ignore_index=True)
display(tabela_final)

In [6]:
teste = tabela_final.copy()
teste['Nome'] = teste['Nome'].apply(unidecode)

In [7]:
teste.head(10)

,Codigo,Nome
0,CIC0004,ALGORITMOS E PROGRAMACAO DE COMPUTADORES
1,FGA0161,ENGENHARIA E AMBIENTE
2,FGA0163,INTRODUCAO A ENGENHARIA
3,FGA0168,DESENHO INDUSTRIAL ASSISTIDO POR COMPUTADOR
4,MAT0025,CALCULO 1
5,DEG0204,VIGILANCIA EPIDEMIOLOGICA COMUNITARIA E PARTIC...
6,FGA0084,DESENVOLVIMENTO DE SOFTWARE
7,FGA0133,ENGENHARIA ECONOMICA
8,FGA0157,PROBABILIDADE E ESTATISTICA APLICADO A ENGENHARIA
9,FGA0158,ORIENTACAO A OBJETOS


In [8]:
def dataframe_to_prolog(df, database_name):
    prolog_db = []

    for index, row in df.iterrows():
        codigo = row['Codigo']
        nome = row['Nome']


        fato = f'disciplina_cursada("{codigo}", "{nome}").'

        prolog_db.append(fato)

    with open(f'{database_name}.pl', 'w') as file:
        for linha in prolog_db:
            file.write(linha + '\n')

In [9]:
dataframe_to_prolog(teste, 'disciplinas_cursadas')